In [1]:
import tqdm
import numpy
import pandas
import scipy.spatial

In [2]:
numpy.random.seed(42)

In [3]:
def initialize_hyperparameters():
    MUTATION_PROBABILITY = 0.1
    CROSSOVER_PROBABILITY = 0.9
    POPULATION_SIZE = 20
    # if K_NEIGHBORS is an odd number, then the algorithm might not work
    K_NEIGHBORS = 10
    GENERATIONS = 100
    return MUTATION_PROBABILITY, CROSSOVER_PROBABILITY, POPULATION_SIZE, GENERATIONS, K_NEIGHBORS

In [4]:
def import_data():
    input_df = pandas.read_csv('u.data', encoding='utf-8', delimiter='\t', engine='python')
    input_df.columns = ['userid', 'movieid', 'ratings', 'timestamp']
    input_df = input_df.drop(columns='timestamp')
    tabular_df_raw = input_df.pivot(index='userid', columns='movieid', values='ratings')
    tabular_df = tabular_df_raw.fillna(0)
    tabular_np = tabular_df.to_numpy()
    return tabular_df, tabular_np

In [5]:
def fetch_user_vector(user, tabular_np):
    user_vector = tabular_np[user][tabular_np[user] != 0]
    user_indeces = numpy.where(tabular_np[user] > 0)
    return user_vector.astype(int), user_indeces

In [6]:
def evaluate_chromosome(chromosome, optim):
    # pearson's correlation metric
#     return numpy.corrcoef(chromosome, optim)[0, 1]
    # cosine similarity metric
#     return 1 - scipy.spatial.distance.cosine(optim, chromosome)
    # mse metric
#     return 10 - (numpy.square(optim - chromosome)).mean(axis=None)
    # number of common elements between chromosomes
    return len(numpy.where(chromosome == optim)[0])

In [7]:
def fetch_neighborhood(user, tabular_np, k):
    user_vector, user_indeces = fetch_user_vector(user, tabular_np)
    data_vector = numpy.take(tabular_np, user_indeces, axis=1).astype(numpy.int64)
    neighbors_data = numpy.zeros(k, dtype=numpy.float64)
    neighbors_indeces = numpy.zeros(k, dtype=int)
    for i in range(data_vector.shape[0]):
        if i != user:
            pearson = evaluate_chromosome(data_vector[i][0], user_vector)
            # check if the given vectors have the same non-zero indeces
            if numpy.where(user_vector > 0)[0].shape[0] == numpy.where(data_vector[i][0] > 0)[0].shape[0]:
                if numpy.all(numpy.where(user_vector > 0)[0] == numpy.where(data_vector[i][0] > 0)[0]):
                    # adjust pearson coefficient to minimum for we do not want that vector to be chosen
                    pearson = -2
                    print('Warning [2]: Same indeces are evaluated [at fetch neighborhood]')
            # sort pearson correlation coefficients
            for a in range(k):
                for b in range(k):
                    if neighbors_data[a] < neighbors_data[b]:
                        tmp = neighbors_data[b]
                        neighbors_data[b] = neighbors_data[a]
                        neighbors_data[a] = tmp
                        tmp = neighbors_indeces[b]
                        neighbors_indeces[b] = neighbors_indeces[a]
                        neighbors_indeces[a] = tmp
            # check if greater pearson coefficient was found
            for j in range(k):
                if neighbors_data[j] < pearson:
                    neighbors_data[j] = pearson
                    neighbors_indeces[j] = i
                    break
    return neighbors_data, neighbors_indeces

In [8]:
def fetch_optim(user, tabular_np, neighbors_indeces):
    optim_indeces = numpy.where(tabular_np[user] == 0)[0]
    for i in neighbors_indeces:
        optim_indeces = numpy.unique(numpy.concatenate((optim_indeces, numpy.where(tabular_np[i] > 0)[0]), 0))
    optim_values = numpy.take(tabular_np, optim_indeces, axis=1)
    return optim_values, optim_indeces

In [9]:
# function that converts parent indeces to the actual population: returns parent pairs [for 10 neighbors: outputs matrix(2x5)]
def index_to_chromosome_decode(idx, population):
    # reshape indeces to handle them easier [with population size = 10, idx is reshaped to: matrix(5,2)]
    idx = idx.reshape(int(population.shape[0]/2), 2)
    # define the generation population
    generation = None
    # convert given indeces to chromosomes 
    for i in range(int(population.shape[0]/2)):
        # extract 2 indeces
        X, Y = population[idx[i]]
        # stack the chromosomes
        if i == 0:
            # if loop run for first time, then initialize the generation population
            generation = numpy.stack((X, Y))
        else:
            # after first time, stack chromosomes to the generation population
            generation = numpy.vstack((generation, numpy.stack((X, Y))))
    # reshape matrix
    population = numpy.zeros((int(generation.shape[0]/2), 2, generation[0].shape[0]), dtype=numpy.int64)
    for i in range(population.shape[0]):
        for j in range(population.shape[1]):
            population[i][j] = generation[i * 2 + j]
    return population

In [10]:
def tournament_selection(population, optim):
    # initialize random sequence
    SEQUENCE = numpy.random.uniform(0, 1, population.shape[0])
    # get pearsons
    pearsons = numpy.empty((population.shape[0]), dtype=numpy.int64)
    pearsons = numpy.fromiter((evaluate_chromosome(population[i], optim) for i in range(population.shape[0])), pearsons.dtype)
    # get parents
    parents = numpy.empty(population.shape[0], dtype=numpy.int64)
    for i in range(population.shape[0]):
        k = numpy.ceil(SEQUENCE[i] * 10).astype(numpy.int64)
        chromosome_pointers = numpy.random.choice(numpy.arange(population.shape[0]), k)
        evaluation = pearsons[chromosome_pointers].max()
        if len(numpy.where(pearsons == evaluation)[0]) > 1:
            parents[i] = numpy.where(pearsons == evaluation)[0][0]
        else:
            parents[i] = numpy.where(pearsons == evaluation)[0]
    return index_to_chromosome_decode(parents, population)

In [11]:
def select_chromosomes(population, optim):
    parent_pairs = tournament_selection(population, optim)
    return parent_pairs

In [12]:
def multiple_point_crossover(parent_pairs, x_probability):
    # define random probability sequence
    SEQUENCE = numpy.random.uniform(0, 1, parent_pairs.shape[0])
    # define new generation population variable
    population_hat = None
    # perform single point crossover 
    for i in range(parent_pairs.shape[0]):
        X, Y = parent_pairs[i]
        # check chromosomes' compatibility in case there is an error
        compatible_chromosomes = X.shape == Y.shape
        # define max index boundary
        chromosome_shape = X.shape[0]
        # initialize new chromosome
        a, b = numpy.zeros((2, chromosome_shape), dtype=numpy.int64)
        if not compatible_chromosomes:
            print("Error [13]: Incompatible chromosomes (at: multiple point selection\nExiting...)")
            exit()
        else:
            # crossover random point
            x_idx, y_idx = numpy.sort(numpy.random.randint(0, chromosome_shape, 2))
            # first child chromosome
            a = numpy.concatenate((X[ :x_idx], Y[x_idx:y_idx], X[y_idx: ]))
            # second child chromosome
            b = numpy.concatenate((Y[ :x_idx], X[x_idx:y_idx], Y[y_idx: ]))
            # crossover with respect to the crosover probability
            if SEQUENCE[i] < x_probability:
                # append children to form the new population
                if i == 0:
                    # if loop run for first time, then initialize the generation population
                    population_hat = numpy.stack((a, b))
                else:
                    # after first time, stack chromosomes to the generation population
                    population_hat = numpy.vstack((population_hat, numpy.stack((a, b))))
            else:
                # append parents to the new population
                if i == 0:
                    # if loop run for first time, then initialize the generation population
                    population_hat = numpy.stack((X, Y))
                else:
                    # after first time, stack chromosomes to the generation population
                    population_hat = numpy.vstack((population_hat, numpy.stack((X, Y))))
    return population_hat

In [13]:
def crossover_chromosomes(x_probability, population, optim):
    parent_pairs = select_chromosomes(population, optim)
    population_hat = multiple_point_crossover(parent_pairs, x_probability)
    # check crossover effect over current generation
    if numpy.array_equal(population, population_hat):
        return population
    else:
        return population_hat

In [14]:
# returns index of elite chromosome
def get_elite_chromosome(population, optim):
    idx, val, max_pearson = -1, None, -1
    for i in range(population.shape[0]):
        val = evaluate_chromosome(population[i], optim)
        if max_pearson < val:
            idx = i
            max_pearson = val
    if idx == -1:
        print('Error [11]: Invalid elite chromosome index [at get elite chromosome]\nExiting...')
        exit()
    return idx

In [15]:
def gauss_replacement(chromosome):
    # define number of genes
    idx_interval = chromosome.shape[0]
    # generate random gaussian distribution
    GAUSSIAN = numpy.random.normal(loc=3, scale=2.0, size=chromosome.shape[0]).astype(numpy.int64)
    # fix lower numbers
    GAUSSIAN[GAUSSIAN < 1] = 1
    # fix higher numbers
    GAUSSIAN[GAUSSIAN > 5] = 5
    # define mutated array
    mutated_chromosome = numpy.zeros(idx_interval, dtype=numpy.int64)
    # define random generator for gene mutation decision
    SEQUENCE = numpy.random.uniform(0, 1, idx_interval)
    for i in range(idx_interval):
        # mutate gene with respect to current SEQUENCE probability
        if SEQUENCE[i] > 0.5:
            # probability valid, MUTATE
            mutated_chromosome[i] = GAUSSIAN[i]
        else:
            # probability inadequate, NO MUTATION
            mutated_chromosome[i] = chromosome[i]
    return mutated_chromosome

In [16]:
def elitism(population, m_probability, optim):
    SEQUENCE = numpy.random.uniform(0, 1, population.shape[0])
    mutated_population, mutated_chromosome = None, None
    elite_chromosome_idx = get_elite_chromosome(population, optim)
    for i in range(population.shape[0]):
        chromosome = population[i]
        if SEQUENCE[i] < m_probability and i != elite_chromosome_idx:
            # mutate chromosome
            mutated_chromosome = gauss_replacement(chromosome)
            # append chromosomes to the mutated population
            if i == 0:
                # if loop run for first time, then initialize the generation population
                mutated_population = mutated_chromosome
            else:
                # after first time, stack chromosomes to the generation population
                mutated_population = numpy.vstack((mutated_population, mutated_chromosome))
        else:
            # NO mutation
            # append chromosomes to the mutated population
            if i == 0:
                # if loop run for first time, then initialize the generation population
                mutated_population = chromosome
            else:
                # after first time, stack chromosomes to the generation population
                mutated_population = numpy.vstack((mutated_population, chromosome))
    return mutated_population

In [17]:
def mutate_chromosomes(probability, population, optim):
    population_hat = elitism(population, probability, optim)
    # check mutation effect over current generation
    if numpy.array_equal(population, population_hat):
        return population
    else:
        return population_hat

In [18]:
def fit_genetic_algorithm(M_PROBABILITY, X_PROBABILITY, P_SIZE, N_GEN, K, items):
    for user in tqdm.tqdm_notebook(range(items.shape[0])):
        # fetch user's rating top - k neighbors (k=10)
        neighbors_data, neighbors_indeces = fetch_neighborhood(user, items, K)
        # fetch optimal solution vector
        optim_values, optim_indeces = fetch_optim(user, items, neighbors_indeces)
        # reformat optimal solution vector
        # trim non neighbor users
        optim_values = optim_values[neighbors_indeces]
        # drop zero values
        optim_mean = numpy.count_nonzero([optim_values], axis=1)
        # find mean of all k vectors and flatten optimal solution to a single vector
        optim_values = optim_values.sum(axis=0)
        optim_indeces = optim_indeces[optim_values != 0]
        optim_values = optim_values[optim_values != 0]
        optim_mean = optim_mean[optim_mean != 0]
        optim_values = optim_values / optim_mean
        # round up mean
        optim_values = numpy.ceil(optim_values).astype(numpy.int64)
        # random initialize population (chromosomes)
        chromosomes = numpy.random.choice(numpy.unique(optim_values), (P_SIZE, optim_values.shape[0]))
        # initialize error (evaluation) array
        evaluation = numpy.zeros(N_GEN)
        # fit GA
        for gen in tqdm.tqdm_notebook(range(N_GEN)):
            # select and crossover population
            chromosomes = crossover_chromosomes(X_PROBABILITY, chromosomes, optim_values)
            # mutate chromosomes
            chromosomes = mutate_chromosomes(M_PROBABILITY, chromosomes, optim_values)

In [19]:
def main():
    # hyperparameter definition
    M_PROBABILITY, X_PROBABILITY, P_SIZE, N_GEN, K = initialize_hyperparameters()
    # GA fitting data
    tabular_df, tabular_np = import_data()
    # fit GA
    fit_genetic_algorithm(M_PROBABILITY, X_PROBABILITY, P_SIZE, N_GEN, K, tabular_np)

In [20]:
if __name__ == '__main__':
    main()

/home/andreas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/andreas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/andreas/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/andreas/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]



Warning [2]: Same indeces are evaluated [at fetch neighborhood]


KeyboardInterrupt: 